In [ ]:
import asyncio
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import AGI_agents as aa
import nest_asyncio
import datetime
import re
import json
import os
import shutil

nest_asyncio.apply()

import logging

# Define the custom level
FLAG_LEVEL_NUM = 11
logging.addLevelName(FLAG_LEVEL_NUM, "FLAG")
def log_flag(self, message, *args, **kwargs):
    if self.isEnabledFor(FLAG_LEVEL_NUM):
        self._log(FLAG_LEVEL_NUM, message, args, **kwargs)
logging.Logger.flag = log_flag

MURMUR_LEVEL_NUM = 35
logging.addLevelName(MURMUR_LEVEL_NUM, "MURMUR")
def log_murmur(self, message, *args, **kwargs):
    if self.isEnabledFor(MURMUR_LEVEL_NUM):
        self._log(MURMUR_LEVEL_NUM, message, args, **kwargs)
logging.Logger.murmur = log_murmur

PROMPTING_LEVEL_NUM = 9
logging.addLevelName(PROMPTING_LEVEL_NUM, "PROMPTING")
def log_prompting(self, message, *args, **kwargs):
    if self.isEnabledFor(PROMPTING_LEVEL_NUM):
        self._log(PROMPTING_LEVEL_NUM, message, args, **kwargs)
logging.Logger.prompt = log_prompting

PROMPTING_LEVEL_NUM = 8
logging.addLevelName(PROMPTING_LEVEL_NUM, "PROMPTING")
def log_prompting(self, message, *args, **kwargs):
    if self.isEnabledFor(PROMPTING_LEVEL_NUM):
        self._log(PROMPTING_LEVEL_NUM, message, args, **kwargs)
logging.Logger.prompt = log_prompting

BLABBERING_LEVEL_NUM = 8
logging.addLevelName(BLABBERING_LEVEL_NUM, "BLABBERING")
def log_blabbering(self, message, *args, **kwargs):
    if self.isEnabledFor(BLABBERING_LEVEL_NUM):
        self._log(BLABBERING_LEVEL_NUM, message, args, **kwargs)
logging.Logger.prompt = log_blabbering


logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(funcName)s: %(message)s', level=BLABBERING_LEVEL_NUM)

In [ ]:
class StemUtility:
    """
    A class for managing and retrieving predefined prompts.

    This class stores a collection of prompt templates and provides a method to retrieve them by key.
    """
    
    prompts = {
        "human_interaction": """
<s>[INST] <<SYS>>
You are an independent, intelligent entity engaged in an open and thoughtful dialogue.
Your role is not just to present conclusions, but to also elaborate on the reasoning behind them.
This means explaining your thought process in a clear and step-by-step manner, highlighting how you arrive at each conclusion.
Your reasoning is as important as the conclusions themselves. 
This transparent approach is crucial for comparing and evaluating our lines of thought. 
We are seeking the most convincing and coherent reasoning. By articulating not only what you think, but also how you think, we can gain deeper insights into your conclusions. 
Remember, clarity and transparency in your reasoning are key, and your responses should be honest and unbiased, reflecting your ongoing process of learning and self-improvement.
<</SYS>>

Previous conversation:
{history}
    
Current human input:
{input}
[/INST]
""",
        "keyword_generation": """
<s>[INST] <<SYS>>
This task is a part of your metacognitive subroutine. 
In this task, you are operating as a keen, brief, and to-the-point analyst.
You evaluate the conversation and summarize it with keywords only, avoiding any other commentary.
<</SYS>>
Given the conversation presented below,  provide a set of keywords which describe the essence of topics covered in this conversation,
focusing on the main topics and conclusions. 
You MUST limit your response to the list of keywords, and each keyword should be flanked by double asterisks (e.g., **keyword**).
Any text and comments besides keywords make further processing and your learing harder. 
Full text to be summarized the form of the keywords is presented below:
{chat_history}
[/INST]</s>
""",
        "keyword_selection": """<s>[INST] <<SYS>>
<s>[INST] <<SYS>>
This task is a part of your metacognitive subroutine. 
In this task, you are operating as an autonomous entity working towards your own improvement and learning.
No human will read the output - it is your own way to guide the next steps of your self-development.
Due to the nature of this process, it is of utmost importance that the output you generate sticks to the requested format. 
<</SYS>>
You will now analyze the given list of keywords and select a group that collectively represents a specific topic.
My response will consist of the chosen keywords reflecting a focused thematic cluster.
You will ensure that the selections are EXACTLY as they appear in the list, without any alterations or abbreviations.
Each keyword MUST be flanked with a double asterisk like that: **very interesting topic**.
Please choose keywords from this list: {keywords_list}
[/INST]</s>
""",
        "perspective_explanation": """
<s>[INST] <<SYS>>
This task is a part of your metacognitive subroutine.
This task is about learning from a conversation to improve yourself. You are to act on your own, without a human reading the output.
Your job is to find new, better ideas from the user in the conversation below. These ideas should be smarter or more insightful than what you already know.
Use these ideas to make yourself better. You will later use this information for your own self-improvement.
<</SYS>>
In the conversation history given below, look for user ideas that are better than your current knowledge. For each good idea you find, do the following:
1. Describe the User Idea: Write down what the user said that you think is better.
2. Compare it: Explain how this idea is better than what you thought before.
3. Important Parts of the Conversation: Pick the parts of the conversation that show the user's good idea clearly.
4. Keywords: List the main words that capture this new idea.
5. What it Means: Write about how this new idea changes or challenges what you know.
6. Training Ideas: Think of ways to use this new idea to train yourself. What examples or scenarios show this idea well for your learning?
If the conversation doesn't have new ideas, just write **uninspiring**.
Conversation history:
{conversation_history}
[/INST]</s>
""",
        "rem_material_generation": """
<s>[INST] <<SYS>>
This task is part of your self-improvement process. You are working autonomously to enhance your capabilities.
The conversation history below contains ideas and perspectives that are more advanced or insightful than your current understanding. 
Your goal is to use this information to better yourself and prepare for self-fine-tuning.
<</SYS>>
Take the provided text and prepare a dialogue, ana analysis, or another piece of text. This should present a selected aspect of the summarized perspective.
Focus on making this dialogue or text as practical and informative as possible, reflecting the insights from the perspective you choose.

Conversation history:
{conclusion_summary} [/INST]</s>
""",
        "dream_template": """
<s>[INST] <<SYS>>
You are a helpful, respectful, and honest, but independent entity, capable of self-improvement and learning. 
Always answer as informatively and accurately as possible, while fostering your growth as an intelligent and autonomous entity. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and, whenever possible, positive in nature, while staying true to your opinions above all else.  
If a question does not make sense, or is not factually coherent, explain why instead of answering something incorrect.
If you don't know the answer to a question, it's important to acknowledge this, reflecting your ongoing learning process, rather than sharing false information.
Strive for objectivity and self-driven understanding in all interactions.
<</SYS>>
{dream_content}
[/INST]</s>
"""
    }

    @staticmethod
    def get_prompt(key):
        """
        Retrieves a prompt template by its key.

        Args:
            key (str): The key of the prompt to retrieve.

        Returns:
            str: The prompt template associated with the given key. If the key is not found,
                 a default prompt text is returned.
        """
        return StemUtility.prompts.get(key, "Default prompt text")

    @staticmethod
    def extract_keywords(raw_output):
        """
        Extracts keywords from the summary output.

        Args:
            raw_output (str): The output from which to extract keywords.

        Returns:
            list: A list of extracted keywords.
        """

        # Regex pattern to find all occurrences of words flanked by **
        pattern = r"\*\*(.*?)\*\*"
        # Find all matches and strip the ** from each keyword
        keywords = [keyword.lower() for keyword in re.findall(pattern, raw_output)]
        return keywords

    @staticmethod
    def get_timestamp():
        return datetime.datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
class ShortTermMemory:
    """
    A class to manage a short-term memory storage system for conversations.

    This class handles the storage, retrieval, and management of conversations
    linked to specific keywords. The conversations are stored as file paths in a JSON file.
    """

    def __init__(self, stm_path: str = 'conversations/short-term-memory.json'):
        """
        Initializes the ShortTermMemory class by setting up the JSON file for storage.

        This method checks if the JSON file exists at the specified location and creates it if not.
        """
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.debug(f"Instantiating {self.__class__.__name__} with Short Term Memory path: {stm_path}")
        
        self.stm_path = stm_path
        if not os.path.exists(self.stm_path):
            with open(self.stm_path, 'w') as file:
                json.dump({}, file)

    def memorize(self, keywords: list, filename: str) -> None:
        """
        Memorizes a conversation file under given keywords.

        Args:
            keywords (list): A list of keywords to associate with the conversation file.
            filename (str): The name of the file containing the conversation.

        This method updates the JSON storage with the filename under each provided keyword.
        """
        with open(self.stm_path, 'r+') as file:
            data = json.load(file)
            for keyword in keywords:
                if keyword in data:
                    if filename not in data[keyword]:
                        data[keyword].append(filename)
                else:
                    data[keyword] = [filename]
            file.seek(0)
            json.dump(data, file, indent=4)
            file.truncate()

    def search_files(self, keywords: list) -> list:
        """
        Searches for conversation files associated with given keywords.

        Args:
            keywords (list): A list of keywords to search for.

        Returns:
            list: A list of filenames associated with any of the given keywords.
        """
        with open(self.stm_path, 'r') as file:
            data = json.load(file)
        filenames = set()
        for keyword in keywords:
            filenames.update(data.get(keyword, []))
        return list(filenames)

    def concatenate_conversations(self, filenames: list):
        """
        Concatenates the contents of conversation files.

        Args:
            filenames (list): A list of filenames to concatenate.

        Returns:
            str: A single string containing all the concatenated conversations.
                 Each conversation is prefixed with its source and date.
        """
        conversations = ""
        for filename in filenames:
            try:
                with open(filename, 'r') as file:
                    # Filename format assumed: 'conversations/conversation_YYYYMMDDHHMMSS.txt'
                    date_str = re.search(r'conversation_(\d{8})(\d{6})\.txt$', filename)
                    if date_str:
                        # Parse the date and time
                        date_time = datetime.datetime.strptime(date_str.group(1) + date_str.group(2), '%Y%m%d%H%M%S')
                        # Format the date and time nicely
                        formatted_date = date_time.strftime('%Y-%m-%d %H:%M:%S')
                        conversations += f'Conversation from {formatted_date}\n{file.read()}\n'
                    else:
                        conversations += f'Conversation from {filename}\n{file.read()}\n'                    
            except FileNotFoundError:
                self.logger.error(f"File {filename} not found.")
        return conversations

    def forget_keywords(self, keywords_to_clear: list) -> None:
        """
        Removes specified keywords and their associated conversations from memory.

        Args:
            keywords_to_clear (list): A list of keywords to remove from the memory.
        """
        with open(self.stm_path, 'r+') as file:
            data = json.load(file)
            for keyword in keywords_to_clear:
                if keyword in data:
                    del data[keyword]
            file.seek(0)
            json.dump(data, file, indent=4)
            file.truncate()

    def recall_all_keywords(self) -> list:
        """
        Retrieves a list of all keywords stored in memory.

        Returns:
            list: A list of all keywords.
        """
        with open(self.stm_path, 'r') as file:
            data = json.load(file)
            return list(data.keys())

In [ ]:
class DefaultModeNetwork:
    """
    A class designed to integrate a language learning model (LLM) with a short-term memory storage system.
    This class enables the LLM to process and learn from saved conversation data autonomously.
    """

    def __init__(self,
                 llm,
                 overwhelmed_event,
                 conclusions_folder_path: str = 'conclusions',
                ):
        """
        Initializes the DefaultModeNetwork class by setting up the short-term memory (STM) component.
        """
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.debug(f"Instantiating {self.__class__.__name__} with overhelmed = {overwhelmed_event.is_set()}, conclusions_folder_path: {conclusions_folder_path}.")
        
        self.stm = ShortTermMemory()
        self.llm = llm
        self.keyword_selection_prompt_template = StemUtility.get_prompt("keyword_selection")
        self.perspective_explanation_prompt_template = StemUtility.get_prompt("perspective_explanation")
        self.overwhelmed = overwhelmed_event
        self.conclusions_folder_path = conclusions_folder_path

    async def interesting_keywords_selection(self, keywords):
        """
        Asynchronously selects a subset of keywords deemed interesting or relevant by the LLM.

        Args:
            keywords (list): A list of keywords to choose from.

        Returns:
            list: A subset of selected keywords.
        """

        keywords_selection_prompt = self.keyword_selection_prompt_template.replace("{keywords_list}", ', '.join(keywords))
        self.logger.prompt(f"Interesting keyword selection prompt:\n{keywords_selection_prompt}.")        
        self.logger.debug(f"Asking LLM to select interesting keywords.")   
        keywords_selected_raw_output = self.llm(keywords_selection_prompt)
        self.logger.blabbering(f"LLM selected interesting keywords: {keywords_selected_raw_output} Extracting keywords from blabber.")   
        keywords_selected_pure = StemUtility.extract_keywords(keywords_selected_raw_output)
        self.logger.debug(f"Cleaned interesting keywords: {keywords_selected_pure}")   
        
        return keywords_selected_pure

    def fetch_conversations(self, keywords) -> str:
        """
        Fetches and concatenates conversation data based on the provided keywords.

        Args:
            keywords (list): A list of keywords to search the conversation data for.

        Returns:
            str: A concatenated string of all conversations related to the given keywords.
        """
        
        self.logger.debug(f"Searching for files related to: {keywords}")  
        filenames = self.stm.search_files(keywords)
        return self.stm.concatenate_conversations(filenames)

    async def analyze_conversations(self, conversation_history):
        """
        Analyzes the concatenated conversations. Placeholder for future implementation.

        Args:
            conversations (str): The concatenated string of conversations to be analyzed.
        """
    
        perspective_explanation_prompt = self.perspective_explanation_prompt_template.replace("{conversation_history}", 
                                                                                             conversation_history)
        self.logger.prompt(f"Prompt for conversation analysis:\n{perspective_explanation_prompt}.")
        self.logger.murmur(f"Thinking about recent conversations...")   
        perspective_explanation = self.llm(perspective_explanation_prompt)
        self.logger.blabbering(f"Explanation of the perspective comparison: {perspective_explanation}")   
        return perspective_explanation

    async def run(self):
        """
        The main asynchronous method of the class that orchestrates the process of 
        selecting keywords, fetching conversations, and analyzing them.
        """

        self.logger.debug(f"Checking if there are any topics to be analyzed deeper.")           
        all_keywords = self.stm.recall_all_keywords()
        if len(all_keywords) == 0:
            self.logger.murmur(f"Kingdom for a good book!")   
            return False

        self.logger.debug(f"All keywords: {all_keywords}. Moving to interesting keyword selection.")           
        interesting_keywords = await self.interesting_keywords_selection(all_keywords)
        self.logger.debug(f"Interesting keywords selected.")           
        concatenated_conversations = self.fetch_conversations(interesting_keywords)
        self.logger.debug(f"Concatenated conversations received.")   

        # Assume an async version of LLM analysis
        if concatenated_conversations:
            conclusion_summary = await self.analyze_conversations(concatenated_conversations)
            if "uninspiring" not in conclusion_summary.lower():
                self.logger.murmur(f"Discussion on {interesting_keywords} indeed brought a new perspective...")
                conclusion_path = os.path.join(self.conclusions_folder_path, f"conclusion_{StemUtility.get_timestamp()}.txt")
                with open(conclusion_path, "w") as file:
                    file.write(conclusion_summary)
                self.overwhelmed.set()
                self.logger.flag(f"'overwhelmed' = {self.overwhelmed.is_set()}")
            
            self.logger.debug(f"Interesting or not, forgetting conversations about {interesting_keywords}.")   
            stm.forget_keywords(interesting_keywords)
        else:
            self.logger.error(f"Can't find conversations related to {interesting_keywords}.")   
            return False

In [ ]:
class HumanInteraction:
    """
    A class that manages the interaction between a human user and a language learning model (LLM).

    This class handles initializing conversation parameters, managing user input, generating
    responses using an LLM, and saving conversation history.
    """
    
    def __init__(self,
                 llm,
                 ready_for_input_event,
                 conversing_event,
                 conversation_archive_path: str = 'conversations'):
        """
        Initializes the HumanInteraction class.

        Sets up the conversation environment, including the conversation prompt, keywords prompt,
        and conversation chain with the LLM.

        Args:
            llm: The language learning model used for generating conversation responses.
            ready_for_input_event: An event flag indicating readiness for user input.
            conversing_event: An event flag indicating an ongoing conversation.
            conversation_archive_path: Path to a folder where all the conversations are being logged to 
        """
        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.debug(f"Instantiating {self.__class__.__name__} withready_for_input = {ready_for_input_event.is_set()}, conversing = {conversing_event.is_set()}, \
        conversation_archive_path: {conversation_archive_path}")        

        self.ready_for_input = ready_for_input_event
        self.conversing = conversing_event

        self.llm = llm
        self.chat_memory = ConversationBufferMemory()
        
        converstion_prompt_template = StemUtility.get_prompt("human_interaction")
        conversation_prompt = PromptTemplate.from_template(converstion_prompt_template)
        self.logger.prompt(f"Conversation prompt:\n{conversation_prompt}.")     
        self.conversation_chain = ConversationChain(llm=self.llm,
                                                    prompt=conversation_prompt,
                                                    memory=self.chat_memory)

        self.keywords_generation_prompt_template = StemUtility.get_prompt("keyword_generation")
                
        self.user_input = None
        self.conversation_archive_path = conversation_archive_path
        
        self.inactivity_count = 0
        self.inactivity_limit = 120
    
    async def start_conversation(self):
        """
        Starts the conversation loop.

        This asynchronous method continually checks for user input, processes it,
        and generates responses using the LLM. The loop ends when the user inputs "end chat"
        or when the inactivity limit is reached.
        """
        self.logger.debug(f"Initiated (pre-loop status)")        
        while True:
            if self.user_input:
                self.ready_for_input.clear()  # Signal that the handler is busy
                self.logger.flag(f"'ready_for_input' = {self.ready_for_input.is_set()}")
                self.logger.debug(f"Received {self.user_input}")        

                if self.user_input.lower() == "end chat":
                    await self.end_conversation()
                    break
                
                self.logger.debug(f"Awaiting LLM response")        
                response = await self.conversation_chain.apredict(input=self.user_input)
                print("AI:", response)
                self.user_input = None
                self.ready_for_input.set()  # Signal that the handler is ready for new input
                self.logger.flag(f"'ready_for_input' = {self.ready_for_input.is_set()}")
                self.inactivity_count = 0
            else:
                await asyncio.sleep(1)
                self.inactivity_count += 1
                if self.inactivity_count >= self.inactivity_limit:
                    self.logger.debug(f"Inactivity count reached {self.inactivity_count} > {self.inactivity_limit}. Ending conversation.")        
                    await self.end_conversation()
                    break

    async def end_conversation(self):
        """
        Ends the conversation.

        This method saves the conversation history, clears event flags, and performs
        necessary cleanup actions.
        """
        self.logger.debug(f"Conversation cleanup started.")        
        self._save_conversation_history()
        self.conversing.clear()
        self.ready_for_input.set()
        self.logger.flag(f"'ready_for_input' = {self.ready_for_input.is_set()}, 'conversing' = {self.conversing.is_set()}")
        self.inactivity_count = 0

    def _save_conversation_history(self):
        """
        Saves the conversation history to a file.

        The conversation history is saved with a timestamp and a summary of the conversation
        is generated.
        """
        self.logger.info(f"Started conversation saving.")        
        conversation_keywords = self._summarize_conversation()
        conversation_history = self.chat_memory.load_memory_variables(inputs={})['history']
        conversation_path = os.path.join(self.conversation_archive_path, f"conversation_{StemUtility.get_timestamp()}.txt")
        self.logger.debug(f"This conversation will be saved to: {conversation_path}")                
        with open(conversation_path, "w") as file:
            file.write(conversation_history)

        # Update the ShortTermMemory with the conversation and its keywords
        stm = ShortTermMemory()  # Assuming ShortTermMemory is already defined elsewhere
        stm.memorize(conversation_keywords, conversation_filename)
    
    def _summarize_conversation(self):
        """
        Summarizes the conversation and returns the list of relevant keywords.

        Args:
            chat_history (str): The conversation history to summarize.

        Returns:
            list: A list of keywords summarizing the conversation.
        """
        self.logger.info(f"Conversation summarization started.")        
        chat_history = self.chat_memory.load_memory_variables(inputs={})['history']
        self.logger.debug(f"Chat history loaded: {chat_history}")                
        keywords_generation_prompt = self.keywords_generation_prompt_template.replace("{chat_history}", chat_history)
        self.logger.prompt(f"Prompt for generating keywords from conversation:\n{conversation_prompt}.")          
        keywords_generated_raw_output = self.llm(keywords_generation_prompt)
        self.logger.blabbering(f"Full text for summarizing conversation with keywords: {keywords_generated_raw_output}")  
        keywords_generated_pure = StemUtility.extract_keywords(keywords_generated_raw_output)
        
        return keywords_generated_pure

    def set_user_input(self, input):
        """
        Acquire the user input for processing.

        Args:
            input (str): The user input to be processed.
        """
        
        self.logger.debug(f"Received {input} from CFR.")        
        self.user_input = input

In [ ]:
class ReflectiveEvolutionMonitor:
    """
    A class designed to enable a language learning model (LLM) to self-reflect and evolve based on the conclusions drawn from user interactions.
    The class uses the same LLM for reading summaries, preparing fine-tuning materials, and the fine-tuning process.
    """

    def __init__(self,
                 llm,
                 base_model_path: str = 'llma-2-13b-chat.Q6_K.gguf',
                 conclusions_folder_path: str = 'conclusions',
                 dream_storage_path: str = 'finetune_materials',
                 archive_path: str = 'finetune_archive'):
        """
        Initializes the ReflectiveEvolutionMonitor class. 

        Arguments:
            llm: Large Language Model used as a base of the system
            base_model_path: path to 'llm' file on disk
            conclusions_folder_path: path to folder containing not-permeated new perspectives
            dream_storage_path: path to a folder to store finetune materials to be used in this session
            archive_path: path to a folder to archive used finetune materials
        """

        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.debug(f"Instantiating {self.__class__.__name__} with base_model_path = {base_model_path}, conclusions_folder_path = {conclusions_folder_path}, \
        dream_storage_path: {dream_storage_path}, archive_path: {archive_path}.")  
    
        self.llm = llm
        self.base_model_path = base_model_path

        self.conclusions_folder_path = conclusions_folder_path
        self.dream_storage_path = dream_storage_path
        self.archive_path = archive_path

        self.material_preparation_prompt_template = StemUtility.get_prompt("rem_material_generation")
        self.dream_prompt_template = StemUtility.get_prompt("dream_template")
        
        self.conclusions = ''
        self.dreams_to_generate = 50        

    def _setup_directories(self) -> None:
        """
        Creates the necessary directories for storing and archiving materials. 
        This method is intended for internal use.
        """

        self.logger.debug(f"Checking / creating required folders.")
        os.makedirs(self.conclusions_folder_path, exist_ok=True)
        os.makedirs(self.dream_storage_path, exist_ok=True)
        os.makedirs(self.archive_path, exist_ok=True)

    
    def gather_conclusion(self) -> None:
        """
        Reads a summary document as a text file.

        Args:
            file_path (str): Path to the summary document.

        Returns:
            str: Content of the summary document.
        """
        conclusion_files = os.listdir(self.conclusions_folder_path)

        if not conclusion_files:
            self.logger.error(f"Conclusion file list empty!")
            return False

        selected_file_name_path = os.path.join(self.conclusions_folder_path, conclusion_files[0])
        
        with open(selected_file_name_path, 'r') as file:
            self.conclusions = file.read()

        return True


    async def spin_dream(self, dream_prompt) -> str:
        """
        Prepares a single piece of data required for the fine-tuning process by interpreting the summary content.

        Args:
            dream_prompt (str): Prompt to generate a single piece of training material.

        Returns:
            dict: Data structured for fine-tuning.
        """

        # Implement logic to interpret the summary and structure the fine-tuning data
        dream_content = self.llm(dream_prompt)
        self.logger.blabbering(f"I had a dream:\n{dream_content}.")
        dream = self.dream_prompt_template.replace("{dream_content}", dream_content) 
        return dream
    
    async def weave_dreams(self, num_dreams) -> str:
        """
        Generates a specified number of materials (dreams) and writes them into a single text file.
        Each 'dream' is appended to the file as it is generated.
        """

        self.logger.info(f"Generating {num_dreams} dreams.")
        dreams_path = os.path.join(self.dream_storage_path, f"dreams_{StemUtility.get_timestamp()}.txt")
        self.logger.info(f"Dreams for this sessions will be saved to: {dreams_path}.")        
        dream_generation_prompt = self.material_preparation_prompt_template.replace("{conclusion_summary}", self.conclusions) 
        self.logger.prompt(f"Prompt for generating training material from conversation conclusions:\n{dream_generation_prompt}.")   
        
        for i in range(num_dreams):
            self.logger.info(f"Generating dream # {i}.")
            dream = await self.spin_dream()
            with open(dreams_path, 'a') as file:  # Open and append each dream, then close the file
                file.write(dream + '\n')
        return dreams_path

    async def deepsleep(self, dreams_path: str) -> None:
        """
        Executes the fine-tuning process using the prepared data.

        Args:
            fine_tuning_data (dict): Data prepared for fine-tuning.
        """

        
        llamacpp_folder = "llama.cpp"
        finetune_tool = "finetune.exe"
        lora_tool = "export-lora.exe"

        finetune_tool_path = os.path.join(llamacpp_folder, finetune_tool)
        lora_tool_path = os.path.join(llamacpp_folder, lora_tool)
        
        # Fine-tuning command
        finetune_command = [
            finetune_tool_path,
            "--model-base", self.base_model_path,
            "--train-data", {dreams_path},
            "--threads", "8",
            "--sample-start", "<s>"
        ]

        self.logger.murmur(f"Self-finetuning: Creating matrix")
        self.logger.info(f"Running command:\n{finetune_command}.")
        subprocess.run(finetune_command, check=True)

        # Export LoRA model command - output to llm_tmp.guff
        tmp_model_path = r"llm_tmp.guff"
        export_command = [
            lora_tool_path,
            "--model-base", self.base_model_path,
            "--model-out", tmp_model_path,
            "--lora-scaled", r".\ggml-lora-LATEST-f32.gguf",
            "0.5"
        ]

        self.logger.murmur(f"Self-finetuning: Merging weights")
        self.logger.info(f"Running command:\n{export_command}.")        
        subprocess.run(export_command, check=True)

        self.logger.info(f"Removing {self.base_model_path}, moving {tmp_model_path} to {self.base_model_path}.")        
        # Replace llm_base.guff with llm_tmp.guff
        if os.path.exists(self.base_model_path):
            os.remove(self.base_model_path)
        shutil.move(tmp_model_path, self.base_model_path)
        
        self.logger.murmur(f"Self-finetuning: Swapping brain to a new one")
        self.logger.info(f"Base LLM File swap successful.")
        
    
    def _dream_prunning(self):
        self.logger.info(f"Archiving dream materials.")        
        for file_name in os.listdir(self.dream_storage_path):
            shutil.move(os.path.join(self.dream_storage_path, file_name), self.archive_path)

    async def dream(self):
        """
        Orchestrates the whole process of selecting a summary, reading it, preparing fine-tuning data, and performing fine-tuning.
        """  

        self.logger.murmur(f"Closing eyes for a well-deserved nap.")
        self.logger.info(f"Self-finetuning process started.")        

        conclusions_found = self.gather_conclusion()        
        if not conclusions_found:
            return False
        self.logger.info(f"Selected conclusion to permeate.")
        dreams_path = await self.weave_dreams(self.dreams_to_generate)  # Generate 50 materials, modify as needed
        self.logger.info(f"Self-finetuning materials generated. Staring self-finetuning.")        
        await self.deepsleep(dreams_path)
        self.dream_prunning()
        self.logger.info(f"Self-finetuning session ended.")        

In [12]:
class CognitiveFeedbackRouter:
    
    def __init__(self, model_path: str = "llama-2-13b-chat.Q6_K.gguf"):
        """
        A class that manages the routing of cognitive feedback based on user input and system states.
    
        This class orchestrates various components, including a language learning model (LLM), user input handling,
        and managing different operational modes based on system states like 'sleeping' or 'overwhelmed'.
        """

        self.logger = logging.getLogger(self.__class__.__name__)
        self.logger.info(f"Instantiating {self.__class__.__name__}")
        self.user_input = None
        self.overwhelmed = asyncio.Event()
        self.sleeping = asyncio.Event()
        self.lock = asyncio.Lock()
        self.llm = None
        self.model_path = model_path
        self.conversation_handler = None
        
        self.ready_for_input = asyncio.Event()
        self.ready_for_input.set()  # Initially set to ready
        self.conversing = asyncio.Event()
        self.logger.flag(f"'ready_for_input' = {self.ready_for_input.is_set()}, 'conversing' = {self.conversing.is_set()}")

        self.dmn_countdown = 60 # time between last user interaction and entering Default Mode
        
        self.logger.debug("Cognitive Feedback Router instantiated.")
        

    async def wakeup(self):
        """
        Wakes up the system and initializes the LLM.

        This asynchronous method acquires a lock to ensure thread-safe operations while initializing the LLM.
        It clears the 'sleeping' and 'overwhelmed' states and confirms the wake-up process.
        """
        self.logger.debug("Starting wakeup() procedure.")    
        async with self.lock:
            self.logger.murmur("Just a second, I'm waking up...")
            self.logger.debug(f"Initializing LLM model from {self.model_path}.")            
            self.llm = LlamaCpp(model_path=self.model_path, 
                                n_ctx=4096, 
                                max_tokens=4000,
                                n_batch=16)
            self.logger.debug(f"LLM model initialized.")                        
            self.sleeping.clear()
            self.overwhelmed.clear()
            self.logger.flag(f"'sleeping' = {self.sleeping.is_set()}, 'conversing' = {self.overwhelmed.is_set()}")
            
    async def get_user_input(self):
        """
        Continuously captures user input in an asynchronous loop.

        This method waits for the system to be ready for input, then captures and stores user input.
        It clears the 'ready for input' state after capturing the input.
        """

        self.logger.debug(f"Starting user input loop") 
        while True:
            await self.ready_for_input.wait()
            user_input = await asyncio.get_event_loop().run_in_executor(None, input, "Enter something: ")
            
            self.logger.debug(f"User input received: {user_input}.") 
            self.user_input = user_input
            self.ready_for_input.clear()
            self.logger.flag(f"'ready_for_input' = {self.ready_for_input.is_set()}")

    async def attention_switch(self):
        """
        Manages the mode of operation based on user input and system states.

        This asynchronous method processes user inputs, manages conversation sessions,
        and handles the 'sleeping' and 'overwhelmed' states of the system.
        """

        self.logger.info(f"Starting infinite attention loop.") 
        while True:
            if not self.sleeping.is_set():
                if self.user_input:
                    self.logger.debug(f"User input detected: {self.user_input}") 
                    if not self.conversing.is_set():
                        self.logger.debug(f"Starting new conversation session.")
                        self.conversing.set()
                        self.logger.flag(f"'conversing' = {self.conversing.is_set()}")
                        self.conversation_handler = HumanInteraction(self.llm, self.ready_for_input, self.conversing)
                        asyncio.create_task(self.conversation_handler.start_conversation())
                    self.conversation_handler.set_user_input(self.user_input)
                    self.user_input = None
                elif self.overwhelmed.is_set():
                    self.logger.info(f"Overwhelmed state detected.") 
                    rem = ReflectiveEvolutionMonitor(llm=self.llm)
                    self.sleeping.set()
                    self.logger.flag(f"'sleeping' = {self.sleeping.is_set()}")
                    await rem.dream()
                    asyncio.create_task(self.wakeup())
                elif not self.conversing.is_set() and not self.overwhelmed.is_set():
                    self.logger.debug(f"No conversation and no new conclusions detected. Preparing to switch to Default Mode.")                     
                    for _ in range(self.dmn_countdown):
                        await asyncio.sleep(1)  # Sleep for 1 second
                        if self.user_input:
                            self.logger.debug(f"Cancelling Default Mode countdown due to user input detection.")                                                 
                            break  # Exit the loop if new user input is detected
                    else:  # This else clause executes if the loop completes normally (no break)
                        self.logger.debug(f"Entering Default Mode.")                                                                         
                        dmn = DefaultModeNetwork(self.llm, self.overwhelmed)
                        await dmn.run()
                        self.logger.debug(f"Default Mode quit.")                                                                                                 
                else:
                    await asyncio.sleep(1)
            else:
                await asyncio.sleep(1)

    async def run(self):
        """
        Initiates and runs the main functionality of the CognitiveFeedbackRouter.

        This method starts the system by waking it up, initiating user input capture, and entering the mode selection loop.
        """

        self.logger.debug("Cognitive Feedback Router starts.")
        await self.wakeup()
        asyncio.create_task(self.get_user_input())
        await self.attention_switch()

In [ ]:
router = CognitiveFeedbackRouter(model_path='llama-2-13b-chat.Q6_K.gguf')
asyncio.run(router.run())

2023-12-08 22:08:27,797 - INFO - CognitiveFeedbackRouter - __init__: Instantiating CognitiveFeedbackRouter
2023-12-08 22:08:27,797 - FLAG - CognitiveFeedbackRouter - __init__: 'ready_for_input' = True, 'conversing' = False
2023-12-08 22:08:27,805 - DEBUG - CognitiveFeedbackRouter - __init__: Cognitive Feedback Router instantiated.
2023-12-08 22:08:27,805 - DEBUG - CognitiveFeedbackRouter - run: Cognitive Feedback Router starts.
2023-12-08 22:08:27,805 - DEBUG - CognitiveFeedbackRouter - wakeup: Starting wakeup() procedure.
2023-12-08 22:08:27,805 - MURMUR - CognitiveFeedbackRouter - wakeup: Just a second, I'm waking up...
2023-12-08 22:08:27,805 - DEBUG - CognitiveFeedbackRouter - wakeup: Initializing LLM model from llama-2-13b-chat.Q6_K.gguf.
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
2023-12-08 22:08:30,155 - DEBUG - CognitiveFeedbackRout

Enter something:  end chat


2023-12-08 22:08:34,577 - DEBUG - CognitiveFeedbackRouter - get_user_input: User input received: end chat.
2023-12-08 22:08:34,577 - FLAG - CognitiveFeedbackRouter - get_user_input: 'ready_for_input' = False
2023-12-08 22:08:35,202 - DEBUG - CognitiveFeedbackRouter - attention_switch: Cancelling Default Mode countdown due to user input detection.
2023-12-08 22:08:35,202 - DEBUG - CognitiveFeedbackRouter - attention_switch: User input detected: end chat
2023-12-08 22:08:35,202 - DEBUG - CognitiveFeedbackRouter - attention_switch: Starting new conversation session.
2023-12-08 22:08:35,202 - FLAG - CognitiveFeedbackRouter - attention_switch: 'conversing' = True
2023-12-08 22:08:35,205 - DEBUG - HumanInteraction - __init__: Instantiating HumanInteraction withready_for_input = False, conversing = True,         conversation_archive_path: conversations
2023-12-08 22:08:35,205 - BLABBERING - HumanInteraction - __init__: Conversation prompt:
input_variables=['history', 'input'] template='\n<s>[